In [ ]:
from fundar_llms.api.ollama import OllamaResponse
import fundar.json
from glob import glob
import polars as pl


def load_and_process(path: str):
    return process_response(fundar.json.load(path))

def process_response(data: dict):
    data.pop('context')
    data['extra'].pop('options')
    data['presidente'] = data['extra'].pop('presidente')
    data['ran_on'] = data['extra'].pop('run_on').replace('_ADDRESS', '')
    data['qid'] = data['extra'].pop('qid')

    assert 0 == len(data['extra'])

    data.pop('extra')

    return data

In [ ]:
responses = glob('../data/responses/q?-*-*.json')
responses = map(load_and_process, responses)
responses = list(responses)

In [ ]:
df = pl.DataFrame(responses)

In [ ]:
df.with_columns([
    pl.col('created_at').str.to_datetime().dt.convert_time_zone('America/Argentina/Buenos_Aires')
]).sort('created_at').with_columns([pl.col('created_at').dt.strftime('%Y/%m/%d %H:%M:%S')]).write_excel('../data/responses.xlsx')